# Part 6: Combined Knowledge Sources

In Parts 1-5, you worked with different types of knowledge sources: search indexes (Parts 1-2), SharePoint (Part 3), web sources (Part 4), and blob storage (Part 5). In Part 6, you'll combine **different types** of knowledge sources and create a unified interface that can query across your entire data ecosystem simultaneously.

## Step 1: Load Environment Variables

Run below cell to load the configuration for your Azure resources, choose the **.venv(3.11.9)** environment that is created for you.

This time you'll create a knowledge base that references knowledge sources from all the different types you've explored.

> **⚠️ Troubleshooting**
>
> If code cells get stuck and keep spinning, select **Restart** from the notebook toolbar at the top. If the issue persists after a couple of tries, close VS Code completely and reopen it.

In [ ]:
import os

from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

load_dotenv(override=True) # take environment variables from .env.

# Azure AI Search configuration
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
credential = DefaultAzureCredential()

# Knowledge base name
knowledge_base_name = "combined-knowledge-base"

# Azure OpenAI configuration
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_chatgpt_deployment = os.getenv("AZURE_OPENAI_CHATGPT_DEPLOYMENT", "gpt-4.1")
azure_openai_chatgpt_model_name = os.getenv("AZURE_OPENAI_CHATGPT_MODEL_NAME", "gpt-4.1")

print("Environment variables loaded")

## Step 2: Create Unified Knowledge Base Across Different Source Types

Now, let's create a knowledge base that combines different types of knowledge sources: search indexes (`healthdocs-knowledge-source` and `hrdocs-knowledge-source`) and web sources (`web-knowledge-source`).

Run the code below to query across different source types (your internal indexed data plus external web content) in one unified interface.

In [ ]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import AzureOpenAIVectorizerParameters, KnowledgeBase, KnowledgeBaseAzureOpenAIModel, KnowledgeRetrievalOutputMode, KnowledgeSourceReference, WebKnowledgeSource

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)

aoai_params = AzureOpenAIVectorizerParameters(
    resource_url=azure_openai_endpoint,
    deployment_name=azure_openai_chatgpt_deployment,
    model_name=azure_openai_chatgpt_model_name,
)

ks = WebKnowledgeSource(
    name="web-knowledge-source",
    description="Knowledge source for Web"
)
index_client.create_or_update_knowledge_source(knowledge_source=ks)
print(f"Knowledge source '{ks.name}' created or updated successfully.")

knowledge_base = KnowledgeBase(
    name=knowledge_base_name,
    models=[KnowledgeBaseAzureOpenAIModel(azure_open_ai_parameters=aoai_params)],
    knowledge_sources=[
        KnowledgeSourceReference(name="healthdocs-knowledge-source"),
        KnowledgeSourceReference(name="hrdocs-knowledge-source"),
        KnowledgeSourceReference(name="web-knowledge-source"),
    ],
    output_mode=KnowledgeRetrievalOutputMode.ANSWER_SYNTHESIS,
    answer_instructions="Always use a bulleted list format when providing answers. Each bullet should be on a separate line. Prepend the name of the knowledge source to the line you cite",
)

index_client.create_or_update_knowledge_base(knowledge_base)
print(f"Knowledge base '{knowledge_base_name}' created or updated successfully.")

## Step 3: Query Multiple Source Types About Internal Topics

The code below asks two questions in one query: one about the Zava CEO (HR information) and another about health plans (health benefits information). This demonstrates how the combined knowledge base can handle queries that span different internal data sources.

The knowledge base will search across HR documents, health documents, and web sources, then synthesize a comprehensive answer that addresses both questions with appropriate citations from each source type.

In [ ]:
from azure.search.documents.knowledgebases import KnowledgeBaseRetrievalClient
from azure.search.documents.knowledgebases.models import KnowledgeBaseMessage, KnowledgeBaseMessageTextContent, KnowledgeBaseRetrievalRequest, SearchIndexKnowledgeSourceParams, WebKnowledgeSourceParams
from IPython.display import display, Markdown

knowledge_base_client = KnowledgeBaseRetrievalClient(endpoint=endpoint, knowledge_base_name=knowledge_base_name, credential=credential)
web_ks_params = WebKnowledgeSourceParams(
    knowledge_source_name="web-knowledge-source",
    include_references=True,
    include_reference_source_data=True
)
healthdocs_ks_params = SearchIndexKnowledgeSourceParams(
    knowledge_source_name="healthdocs-knowledge-source",
    include_references=True,
    include_reference_source_data=True,
)
hrdocs_ks_params = SearchIndexKnowledgeSourceParams(
    knowledge_source_name="hrdocs-knowledge-source",
    include_references=True,
    include_reference_source_data=True,
)
req = KnowledgeBaseRetrievalRequest(
    messages=[
        KnowledgeBaseMessage(role="user", content=[KnowledgeBaseMessageTextContent(text="""
            What is the responsibility of the Zava CEO?
            What Zava health plan would you recommend if they wanted the best coverage for mental health services?                          
        """)])
    ],
    knowledge_source_params=[
        web_ks_params, healthdocs_ks_params, hrdocs_ks_params
    ],
    include_activity=True
)

result = knowledge_base_client.retrieve(retrieval_request=req)
display(Markdown(result.response[0].content[0].text))

## Step 4: Query Web-Only Topics

The next query demonstrates how the combined knowledge base intelligently routes questions to appropriate sources. When you ask about general world knowledge (like the Eiffel Tower), the system will primarily use the web knowledge source since your internal HR and health documents wouldn't contain this information.

This shows the knowledge base's ability to understand which sources are most relevant for different types of queries, even when all sources are available.

In [ ]:
req = KnowledgeBaseRetrievalRequest(
    messages=[
        KnowledgeBaseMessage(role="user", content=[KnowledgeBaseMessageTextContent(text="""
            How tall is the Eiffel tower?                    
        """)])
    ],
    knowledge_source_params=[
        web_ks_params, healthdocs_ks_params, hrdocs_ks_params
    ],
    include_activity=True
)

result = knowledge_base_client.retrieve(retrieval_request=req)
display(Markdown(result.response[0].content[0].text))

## Summary

You've now worked with combined knowledge sources in a single knowledge base that spans different types of data sources.

**Key concepts to remember:**
- A single knowledge base can reference different types of knowledge sources (search indexes, web sources)
- The knowledge base intelligently routes queries to appropriate sources based on content
- Combined sources provide comprehensive answers with proper source attribution
- The system eliminates the need to manually decide which knowledge source to query

### What's Next?

➡️ Continue to [Part 7: Minimal Knowledge Base](part7-minimal-knowledge-base.ipynb) to learn how to optimize for speed and cost with minimal reasoning effort.